# 132: Stroke Progression + PQTL

Reminder, you can run something like:

`jupyter nbconvert --to html yourNotebook.ipynb`


# Manhattan Plots:

## First AIS (Existing Published GWAS vs. our meta-analysed GWAS)

![image](scratch/results/existing_stroke_gwas.png)
![image](scratch/results/meta_analysis/first_ais_metal_metaanalysis_manhattan.png)

Comparing by eye, they align quite well, with the existing stroke GWAS having much better results due to high power. HOWEVER, taking the top hitting 30 SNP results don't match  well with our results.  Therefore, I downloaded the full summary stats, and am looking at the wider range of hits.

## Subsequent GWASes (meta-analysed only)

![image](scratch/results/meta_analysis/subsequent_ais_metal_metaanalysis_manhattan.png)
![image](scratch/results/meta_analysis/subsequent_mace_metal_metaanalysis_manhattan.png)

## Subsequent GWASes (adjusted for collider bias, then meta-analysed)

add me later

In [12]:
source("gwas_data_helpers.R")
library(dplyr)
options(repr.matrix.max.rows=600, repr.matrix.max.cols=200)
gwas_columns <- c("MARKER", "BETA", "SE", "P")

first_ais_metaanalysis <- data.table::fread("scratch/results/meta_analysis/first_ais_metal_metaanalysis.tbl", select = c(gwas_columns, "CHR", "BP"))
subsequent_ais_metaanalysis_not_adjusted <- data.table::fread("scratch/results/meta_analysis/subsequent_ais_metal_metaanalysis.tbl", select = gwas_columns)
subsequent_mace_metaanalysis_not_adjusted <- data.table::fread("scratch/results/meta_analysis/subsequent_mace_metal_metaanalysis.tbl", select = gwas_columns)
subsequent_ais_metaanalysis_adjusted <- data.table::fread("scratch/results/subsequent_ais_metaanalysis_slopehunter_clumped1.txt", select = gwas_columns)
subsequent_mace_metaanalysis_adjusted <- data.table::fread("scratch/results/subsequent_mace_metaanalysis_slopehunter_clumped1.txt", select = gwas_columns)
subsequent_ais_metaanalysis_dudbridge <- data.table::fread("scratch/results/subsequent_ais_metaanalysis_dudbridge_clumped1.txt", select = gwas_columns)
subsequent_mace_metaanalysis_dudbridge <- data.table::fread("scratch/results/subsequent_mace_metaanalysis_dudbridge_clumped1.txt", select = gwas_columns)

existing_stroke_gwas <- data.table::fread("scratch/data/existing_stroke_gwas.tsv.gz")
existing_stroke_gwas <- existing_stroke_gwas %>% dplyr::rename("CHR" = "chromosome", "BP" = "base_pair_location", "P" = "p_value",
                                                               "BETA" = "beta", "OR" = "odds_ratio", "LB" = "ci_lower", "UB" = "ci_upper")

existing_stroke_gwas$MARKER <- paste(existing_stroke_gwas$CHR, existing_stroke_gwas$BP, sep=":")
existing_stroke_gwas <- convert_odds_ratio_to_beta_estimate(existing_stroke_gwas)
existing_stroke_gwas_top_hits <- existing_stroke_gwas[existing_stroke_gwas$P < 5e-8,]


merge_gwases <- function(gwas) {
    merged_stroke_data_comparison <- merge(gwas, subsequent_ais_metaanalysis_not_adjusted, by = "MARKER")
    merged_stroke_data_comparison <- merged_stroke_data_comparison %>% dplyr::rename("EXISTING_BETA" = "BETA.x", "EXISTING_P" = "P.x", "EXISTING_SE" = "SE.x",
                                                                                     "SUB_AIS_BETA" = "BETA.y", "SUB_AIS_P" = "P.y", "SUB_AIS_SE" = "SE.y")

    merged_stroke_data_comparison <- merge(merged_stroke_data_comparison, subsequent_ais_metaanalysis_adjusted, by = "MARKER")
    merged_stroke_data_comparison <- merged_stroke_data_comparison %>% dplyr::rename("SUB_AIS_SH_BETA" = "BETA", "SUB_AIS_SH_P" = "P", "SUB_AIS_SH_SE" = "SE")
    
    merged_stroke_data_comparison <- merge(merged_stroke_data_comparison, subsequent_ais_metaanalysis_dudbridge, by = "MARKER")
    merged_stroke_data_comparison <- merged_stroke_data_comparison %>% dplyr::rename("SUB_AIS_DB_BETA" = "BETA", "SUB_AIS_DB_P" = "P", "SUB_AIS_DB_SE" = "SE")    

    merged_stroke_data_comparison <- merge(merged_stroke_data_comparison, subsequent_mace_metaanalysis_not_adjusted, by = "MARKER")
    merged_stroke_data_comparison <- merged_stroke_data_comparison %>% dplyr::rename("SUB_MACE_BETA" = "BETA", "SUB_MACE_P" = "P", "SUB_MACE_SE" = "SE")

    merged_stroke_data_comparison <- merge(merged_stroke_data_comparison, subsequent_mace_metaanalysis_adjusted, by = "MARKER")
    merged_stroke_data_comparison <- merged_stroke_data_comparison %>% dplyr::rename("SUB_MACE_SH_BETA" = "BETA", "SUB_MACE_SH_P" = "P", "SUB_MACE_SH_SE" = "SE")
    
    merged_stroke_data_comparison <- merge(merged_stroke_data_comparison, subsequent_mace_metaanalysis_dudbridge, by = "MARKER")
    merged_stroke_data_comparison <- merged_stroke_data_comparison %>% dplyr::rename("SUB_MACE_DB_BETA" = "BETA", "SUB_MACE_DB_P" = "P", "SUB_MACE_DB_SE" = "SE")

    return(merged_stroke_data_comparison)
}

In [52]:
first_ais_metaanalysis <- data.table::fread("scratch/results/meta_analysis/first_ais_metal_metaanalysis.tbl", select = c(gwas_columns, "CHR", "BP"))

## Comparing p-values across all the different GWASes

There are two interesting things to notice from this table:
* In general, the subsequent MACE hits seem to align more closely with the incidence than the subsequent AIS does.
* SlopeHunter correction decimates the p-values of the MACE hits that show promise.



In [53]:
merged_existing_gwas <- merge_gwases(existing_stroke_gwas_top_hits)
merged_existing_gwas <- merge(merged_existing_gwas, first_ais_metaanalysis, by = "MARKER")
merged_existing_gwas <- merged_existing_gwas %>% dplyr::rename("FIRST_BETA" = "BETA", "FIRST_P" = "P", "FIRST_SE" = "SE")

merged_existing_gwas <- transform(merged_existing_gwas, SUB_AIS_MIN_P_VALUE = pmin(SUB_AIS_P, SUB_AIS_SH_P, SUB_AIS_DB_P))
merged_existing_gwas <- transform(merged_existing_gwas, SUB_MACE_MIN_P_VALUE = pmin(SUB_MACE_P, SUB_MACE_SH_P, SUB_MACE_DB_P))

print("Subsequent AIS Top Hits (when filtered by existing incidence GWAS 5e-8)")
pruned_ais_snps <- basic_snp_pruning(merged_existing_gwas, p_value_column="SUB_AIS_MIN_P_VALUE")
pruned_ais_snps[with(pruned_ais_snps, order(SUB_AIS_MIN_P_VALUE)), c("MARKER", "EXISTING_P", "FIRST_P", "SUB_AIS_P", "SUB_AIS_SH_P", "SUB_AIS_DB_P")]
print("Subsequent MACE Top Hits (when filtered by existing incidence GWAS 5e-8)")
pruned_mace_snps <- basic_snp_pruning(merged_existing_gwas, p_value_column="SUB_AIS_MIN_P_VALUE")
pruned_mace_snps[with(pruned_mace_snps, order(SUB_MACE_MIN_P_VALUE)), c("MARKER", "EXISTING_P", "FIRST_P", "SUB_MACE_P", "SUB_MACE_SH_P", "SUB_MACE_DB_P")]

[1] "Subsequent AIS Top Hits (when filtered by existing incidence GWAS 5e-8)"
[1] 0
[1] 1034
[1] 0
[1] 1034
[1] 0
[1] 1034
[1] 0
[1] 1034
[1] 0
[1] 1034
[1] 0
[1] 1034
[1] 0
[1] 1034
[1] 0
[1] 1034
[1] 0
[1] 1034
[1] 0
[1] 1034
[1] 0
[1] 1034
[1] 0
[1] 1034
[1] 0
[1] 1034
[1] 0
[1] 1034
[1] 0
[1] 1034
[1] 0
[1] 1034
[1] 0
[1] 1034
[1] 0
[1] 1034
[1] 0
[1] 1034
[1] 0
[1] 1034
[1] 0
[1] 1034
[1] 0
[1] 1034
[1] 0
[1] 1034
[1] 0
[1] 1034
[1] 0
[1] 1034
[1] 0
[1] 1034
[1] 0
[1] 1034
[1] 0
[1] 1034
[1] 0
[1] 1034
[1] 0
[1] 1034
[1] 0
[1] 1034
[1] 0
[1] 1034
[1] 0
[1] 1034
[1] 0
[1] 1034
[1] 0
[1] 1034
[1] 0
[1] 1034
[1] 0
[1] 1034
[1] 0
[1] 1034
[1] 0
[1] 1034
[1] 0
[1] 1034
[1] 0
[1] 1034
[1] 0
[1] 1034
[1] 0
[1] 1034
[1] 0
[1] 1034
[1] 0
[1] 1034
[1] 0
[1] 1034
[1] 0
[1] 1034
[1] 0
[1] 1034
[1] 0
[1] 1034
[1] 0
[1] 1034
[1] 0
[1] 1034
[1] 0
[1] 1034
[1] 0
[1] 1034
[1] 0
[1] 1034
[1] 0
[1] 1034
[1] 0
[1] 1034
[1] 0
[1] 1034
[1] 0
[1] 1034
[1] 0
[1] 1034
[1] 0
[1] 1034
[1] 0
[1] 1034
[1] 0
[

MARKER,EXISTING_P,FIRST_P,SUB_AIS_P,SUB_AIS_SH_P,SUB_AIS_DB_P
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
7:92243719,3.114e-09,3.784e-01,0.0005098,0.0004415,0.0001885
4:155524009,1.972e-10,8.933e-02,0.0002233,0.0039340,0.0012800
5:121518309,5.642e-11,5.345e-01,0.0002285,0.0936600,0.0453300
6:1364471,5.152e-09,3.910e-04,0.0002929,0.0325700,0.0035280
4:155537601,1.792e-09,1.002e-01,0.0003129,0.0036990,0.0014620
4:155525970,4.293e-10,8.632e-02,0.0003371,0.0045390,0.0016420
5:121522778,4.083e-09,5.504e-01,0.0003607,0.0758300,0.0410200
6:1364691,1.695e-11,1.611e-05,0.0003946,0.0558700,0.0046340
9:22124450,1.167e-11,1.181e-05,0.0004771,0.0053670,0.0004495


[1] "Subsequent MACE Top Hits (when filtered by existing incidence GWAS 5e-8)"
[1] 0
[1] 1034
[1] 0
[1] 1034
[1] 0
[1] 1034
[1] 0
[1] 1034
[1] 0
[1] 1034
[1] 0
[1] 1034
[1] 0
[1] 1034
[1] 0
[1] 1034
[1] 0
[1] 1034
[1] 0
[1] 1034
[1] 0
[1] 1034
[1] 0
[1] 1034
[1] 0
[1] 1034
[1] 0
[1] 1034
[1] 0
[1] 1034
[1] 0
[1] 1034
[1] 0
[1] 1034
[1] 0
[1] 1034
[1] 0
[1] 1034
[1] 0
[1] 1034
[1] 0
[1] 1034
[1] 0
[1] 1034
[1] 0
[1] 1034
[1] 0
[1] 1034
[1] 0
[1] 1034
[1] 0
[1] 1034
[1] 0
[1] 1034
[1] 0
[1] 1034
[1] 0
[1] 1034
[1] 0
[1] 1034
[1] 0
[1] 1034
[1] 0
[1] 1034
[1] 0
[1] 1034
[1] 0
[1] 1034
[1] 0
[1] 1034
[1] 0
[1] 1034
[1] 0
[1] 1034
[1] 0
[1] 1034
[1] 0
[1] 1034
[1] 0
[1] 1034
[1] 0
[1] 1034
[1] 0
[1] 1034
[1] 0
[1] 1034
[1] 0
[1] 1034
[1] 0
[1] 1034
[1] 0
[1] 1034
[1] 0
[1] 1034
[1] 0
[1] 1034
[1] 0
[1] 1034
[1] 0
[1] 1034
[1] 0
[1] 1034
[1] 0
[1] 1034
[1] 0
[1] 1034
[1] 0
[1] 1034
[1] 0
[1] 1034
[1] 0
[1] 1034
[1] 0
[1] 1034
[1] 0
[1] 1034
[1] 0
[1] 1034
[1] 0
[1] 1034
[1] 0
[1] 1034
[1] 0


MARKER,EXISTING_P,FIRST_P,SUB_MACE_P,SUB_MACE_SH_P,SUB_MACE_DB_P
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
9:22114495,1.400e-09,1.170e-04,2.360e-06,0.0002682,1.692e-05
9:22124450,1.167e-11,1.181e-05,3.775e-06,0.0024810,1.852e-04
9:22124630,1.600e-11,1.063e-05,3.887e-06,0.0025680,1.916e-04
9:22115026,6.147e-12,1.093e-04,4.201e-04,0.0002852,1.377e-05
9:22088094,3.831e-13,7.124e-06,1.506e-05,0.0037700,2.792e-04
9:22115286,1.015e-13,1.019e-04,3.444e-04,0.0003790,1.759e-05
9:22116220,2.739e-13,1.013e-04,3.446e-04,0.0003939,1.852e-05
9:22112599,3.713e-13,9.049e-05,2.693e-05,0.0040700,7.463e-04
9:22088260,1.085e-08,7.287e-06,2.950e-05,0.0026290,1.621e-04


Look at the above in comparison with the starting point of the meta analysed incidence GWAS that we have available from our data...

Interesting thing to note here:
* The Dudbridge Correction seemed to increase most of these hits...

In [3]:
significant_first_ais_comparison <- first_ais_metaanalysis[first_ais_metaanalysis$P < 5e-8,]
significant_first_ais_comparison <- merge_gwases(significant_first_ais_comparison)

significant_first_ais_comparison[, c("MARKER", "EXISTING_P", "SUB_AIS_P", "SUB_AIS_SH_P", "SUB_AIS_DB_P", "SUB_MACE_P", "SUB_MACE_SH_P", "SUB_MACE_DB_P")]

MARKER,EXISTING_P,SUB_AIS_P,SUB_AIS_SH_P,SUB_AIS_DB_P,SUB_MACE_P,SUB_MACE_SH_P,SUB_MACE_DB_P
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
14:103199710,8.858e-09,0.880900,0.68320,0.757100,8.008e-01,3.627e-01,7.063e-01
1:182977036,1.675e-08,0.753500,0.05808,0.709500,3.495e-01,2.426e-04,1.355e-01
1:182978124,2.061e-08,0.857400,0.08124,0.829000,3.733e-01,2.712e-04,1.280e-01
1:182979308,2.205e-08,0.787500,0.05734,0.756600,3.505e-01,2.009e-04,1.140e-01
1:182980016,2.272e-08,0.800900,0.06186,0.770400,3.557e-01,2.160e-04,1.147e-01
1:182982597,2.205e-08,0.787500,0.05734,0.756600,3.505e-01,2.009e-04,1.140e-01
1:182983635,2.205e-08,0.787500,0.05734,0.756600,3.505e-01,2.009e-04,1.140e-01
1:182985440,2.132e-08,0.783500,0.06140,0.764100,3.487e-01,2.167e-04,1.199e-01
1:182986808,2.170e-08,0.872500,0.12340,0.744400,5.272e-01,3.138e-04,9.177e-02


In [45]:
source("gwas_data_helpers.R")

floor(125000/2)
testout <- significant_first_ais_comparison
testout <- separate(data = testout, col = "MARKER", into = c("CHR", "BP"), sep = ":", remove = F)
testout$BP <- as.numeric(testout$BP)
testout <- basic_snp_pruning(testout, p_value_column="EXISTING_P", range=125000)
testout

[1] 62500

[1] 0
[1] 89
[1] 18
[1] 71
[1] 24
[1] 47
[1] 2
[1] 45
[1] 6
[1] 39
[1] 9
[1] 30
[1] 3
[1] 27
[1] 5
[1] 22
[1] 2
[1] 20
[1] 4
[1] 16
[1] 3
[1] 13
[1] 3
[1] 10
[1] 0
[1] 10
[1] 0
[1] 10
[1] 0
[1] 10
[1] 0
[1] 10
[1] 0
[1] 10
[1] 0
[1] 10
[1] 0
[1] 10
[1] 0
[1] 10
[1] 0
[1] 10
[1] 0
[1] 10
[1] 0
[1] 10
[1] 0
[1] 10
[1] 0
[1] 10
[1] 0
[1] 10
[1] 0
[1] 10
[1] 0
[1] 10
[1] 0
[1] 10
[1] 0
[1] 10
[1] 0
[1] 10
[1] 0
[1] 10
[1] 0
[1] 10
[1] 0
[1] 10
[1] 0
[1] 10
[1] 0
[1] 10
[1] 0
[1] 10
[1] 0
[1] 10
[1] 0
[1] 10
[1] 0
[1] 10
[1] 0
[1] 10
[1] 0
[1] 10
[1] 0
[1] 10
[1] 0
[1] 10
[1] 0
[1] 10
[1] 0
[1] 10
[1] 0
[1] 10
[1] 0
[1] 10
[1] 0
[1] 10
[1] 0
[1] 10
[1] 0
[1] 10
[1] 0
[1] 10
[1] 0
[1] 10
[1] 0
[1] 10
[1] 0
[1] 10
[1] 0
[1] 10
[1] 0
[1] 10
[1] 0
[1] 10
[1] 0
[1] 10
[1] 0
[1] 10
[1] 0
[1] 10
[1] 0
[1] 10
[1] 0
[1] 10
[1] 0
[1] 10
[1] 0
[1] 10
[1] 0
[1] 10
[1] 0
[1] 10
[1] 0
[1] 10
[1] 0
[1] 10
[1] 0
[1] 10
[1] 0
[1] 10
[1] 0
[1] 10
[1] 0
[1] 10
[1] 0
[1] 10
[1] 0
[1] 10
[1] 0
[1] 10
[1] 0
[1] 

,MARKER,CHR,BP,EXISTING_BETA,EXISTING_SE,EXISTING_P,SUB_AIS_BETA,SUB_AIS_SE,SUB_AIS_P,SUB_AIS_SH_BETA,SUB_AIS_SH_SE,SUB_AIS_SH_P,SUB_AIS_DB_BETA,SUB_AIS_DB_SE,SUB_AIS_DB_P,SUB_MACE_BETA,SUB_MACE_SE,SUB_MACE_P,SUB_MACE_SH_BETA,SUB_MACE_SH_SE,SUB_MACE_SH_P,SUB_MACE_DB_BETA,SUB_MACE_DB_SE,SUB_MACE_DB_P
,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,14:103199710,14,103199710,0.0079,0.0014,8.858e-09,0.0089,0.0592,0.880900,0.0257,0.0629,0.68320,-0.0183,0.0592,0.757100,0.0143,0.0568,8.008e-01,0.0539,0.0592,0.362700,0.0214,0.0569,0.7063000
43,1:183114864,1,183114864,0.0378,0.0059,1.126e-10,-0.0031,0.0108,0.775200,-0.0394,0.0217,0.06926,-0.0064,0.0111,0.565600,-0.0082,0.0110,4.573e-01,0.0734,0.0227,0.001261,0.0125,0.0113,0.2686000
45,2:203644529,2,203644529,-0.0520,0.0092,1.793e-08,-0.0487,0.0172,0.004729,-0.0819,0.0453,0.07059,-0.0463,0.0177,0.008824,-0.0245,0.0175,1.618e-01,-0.0428,0.0455,0.346700,-0.0089,0.0180,0.6197000
49,2:203736202,2,203736202,-0.0504,0.0089,1.484e-08,-0.0443,0.0166,0.007509,-0.0875,0.0467,0.06086,-0.0414,0.0170,0.014950,-0.0213,0.0168,2.049e-01,-0.0350,0.0465,0.451500,-0.0032,0.0172,0.8522000
63,2:203865822,2,203865822,0.0521,0.0089,5.581e-09,0.0424,0.0167,0.010870,0.0823,0.0465,0.07700,0.0383,0.0171,0.025080,0.0205,0.0169,2.267e-01,0.0297,0.0465,0.523100,0.0005,0.0174,0.9774000
68,4:89943023,4,89943023,0.0331,0.0055,1.366e-09,0.0082,0.0101,0.414400,0.0133,0.0202,0.51150,-0.0032,0.0104,0.756700,-0.0009,0.0102,9.317e-01,-0.0195,0.0210,0.350900,-0.0247,0.0105,0.0191200
75,6:1375827,6,1375827,-0.0573,0.0102,2.189e-08,-0.0567,0.0187,0.002453,-0.0125,0.0593,0.83300,-0.0407,0.0192,0.034250,-0.0265,0.0190,1.637e-01,-0.0654,0.0585,0.263500,-0.0120,0.0194,0.5362000
81,7:92263789,7,92263789,0.0376,0.0063,2.594e-09,0.0271,0.0117,0.020800,0.0471,0.0260,0.06938,0.0266,0.0120,0.026960,0.0236,0.0119,4.732e-02,0.0106,0.0265,0.690300,0.0081,0.0122,0.5040000
83,8:10766675,8,10766675,-0.0337,0.0057,3.538e-09,-0.0021,0.0106,0.840500,0.0241,0.0220,0.27320,0.0109,0.0108,0.311100,-0.0090,0.0108,4.016e-01,0.0027,0.0229,0.907500,0.0115,0.0110,0.2931000


## Working Backwards from the Subsequent GWAS Results



In [4]:
existing_stroke_gwas$MARKER <- paste(existing_stroke_gwas$CHR, existing_stroke_gwas$BP, sep=":")
existing_stroke_gwas <- convert_odds_ratio_to_beta_estimate(existing_stroke_gwas)

significant_subsequent_ais_comparison <- subsequent_ais_metaanalysis_not_adjusted[subsequent_ais_metaanalysis_not_adjusted$P < 5e-6,]
significant_subsequent_ais_comparison <- merge_gwases(significant_subsequent_ais_comparison)
significant_subsequent_ais_comparison <- merge(significant_subsequent_ais_comparison, existing_stroke_gwas, by = "MARKER")
significant_subsequent_ais_comparison <- significant_subsequent_ais_comparison %>% dplyr::rename("ORIG_EXISTING_BETA" = "BETA", "ORIG_EXISTING_P" = "P", "ORIG_EXISTING_SE" = "SE")

significant_subsequent_ais_comparison[, c("MARKER", "ORIG_EXISTING_P", "SUB_AIS_P", "SUB_AIS_SH_P", "SUB_AIS_DB_P")]

significant_subsequent_mace_comparison <- subsequent_mace_metaanalysis_not_adjusted[subsequent_mace_metaanalysis_not_adjusted$P < 5e-6,]
significant_subsequent_mace_comparison <- merge_gwases(significant_subsequent_mace_comparison)
significant_subsequent_mace_comparison <- merge(significant_subsequent_mace_comparison, existing_stroke_gwas, by = "MARKER")
significant_subsequent_mace_comparison <- significant_subsequent_mace_comparison %>% dplyr::rename("ORIG_EXISTING_BETA" = "BETA", "ORIG_EXISTING_P" = "P", "ORIG_EXISTING_SE" = "SE")

significant_subsequent_mace_comparison[, c("MARKER", "ORIG_EXISTING_P", "SUB_MACE_P", "SUB_MACE_SH_P", "SUB_MACE_DB_P")]

MARKER,ORIG_EXISTING_P,SUB_AIS_P,SUB_AIS_SH_P,SUB_AIS_DB_P
<chr>,<dbl>,<dbl>,<dbl>,<dbl>
12:108665953,0.3038,4.861e-06,2.191e-05,1.646e-05
12:108666175,0.3560,4.522e-06,2.042e-05,1.537e-05
14:73294871,0.3027,2.290e-06,1.666e-05,6.377e-06
14:73295026,0.2767,4.786e-06,1.445e-04,7.976e-05
14:73301828,0.3843,3.034e-06,3.100e-05,1.200e-05
14:73301903,0.3324,3.011e-06,2.754e-05,1.190e-05
14:73303231,0.3425,4.431e-06,1.333e-04,7.339e-05
16:4411506,0.3004,1.533e-06,6.215e-03,3.461e-03
16:4417180,0.3455,1.538e-06,5.352e-03,3.152e-03


MARKER,ORIG_EXISTING_P,SUB_MACE_P,SUB_MACE_SH_P,SUB_MACE_DB_P
<chr>,<dbl>,<dbl>,<dbl>,<dbl>
13:112784083,9.792e-01,1.007e-06,4.892e-01,4.212e-01
13:62088326,5.779e-01,4.924e-06,1.859e-03,3.627e-03
16:4411506,3.004e-01,7.825e-07,6.294e-02,2.398e-02
16:4417180,3.455e-01,5.453e-07,5.224e-02,1.999e-02
16:4417338,2.605e-01,9.064e-07,5.906e-02,2.250e-02
16:4417797,2.794e-01,4.132e-07,4.926e-02,1.706e-02
16:4419121,3.634e-01,5.981e-07,5.355e-02,1.988e-02
16:4422744,9.477e-02,2.474e-06,6.872e-02,4.014e-02
19:34198233,6.495e-02,4.396e-06,6.797e-03,1.849e-02
